In [41]:
import pandas as pd
import lightgbm as lgb

In [42]:
train_data = pd.read_csv('工业场景下的服装生产力预测挑战赛公开数据/train.csv')
test_data = pd.read_csv('工业场景下的服装生产力预测挑战赛公开数据/test.csv')

In [43]:
train_data.head()

,id,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,...,department_finishing,department_sweing,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,targeted_score,score
0,0,9,20,1305,4816,28,0,0,0,48,...,0,0,0,0,0,0,0,0,70,59
1,1,7,30,831,6544,20,17,1,0,56,...,0,1,0,0,0,0,0,0,65,55
2,2,4,18,994,1089,0,2,21,0,35,...,0,0,0,0,0,0,0,0,80,26
3,3,6,27,810,6635,0,0,0,0,55,...,0,1,0,0,0,0,0,0,63,31
4,4,9,20,1517,5804,0,0,0,0,48,...,0,0,0,0,0,0,0,0,70,35


In [44]:
test_data.head()

,id,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,...,quarter_Quarter5,department_finishing,department_sweing,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,targeted_score
0,0,3,4,1183,2764,0,0,0,0,15,...,0,0,0,0,0,0,0,0,0,46
1,1,2,3,1183,1798,0,0,0,0,9,...,0,0,0,0,0,0,0,0,0,75
2,2,6,13,936,2456,0,0,0,0,18,...,0,0,0,0,0,0,0,0,0,77
3,3,5,19,844,4179,1,0,0,0,34,...,0,0,1,0,1,0,0,0,0,78
4,4,9,3,1183,960,0,0,0,0,8,...,0,0,0,0,0,0,0,0,1,50


In [45]:
train_data.iloc[0]

id                         0
team                       9
smv                       20
wip                     1305
over_time               4816
incentive                 28
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers             48
month                      2
quarter_Quarter1           0
quarter_Quarter2           0
quarter_Quarter3           0
quarter_Quarter4           0
quarter_Quarter5           0
department_finishing       0
department_sweing          0
day_Monday                 0
day_Saturday               0
day_Sunday                 0
day_Thursday               0
day_Tuesday                0
day_Wednesday              0
targeted_score            70
score                     59
Name: 0, dtype: int64

In [46]:
train_data['team_mean_score'] = train_data['team'].map(train_data.groupby('team')['score'].mean())
test_data['team_mean_score'] = test_data['team'].map(train_data.groupby('team')['score'].mean())

train_data['month_mean_score'] = train_data['month'].map(train_data.groupby('month')['score'].mean())
test_data['month_mean_score'] = test_data['month'].map(train_data.groupby('month')['score'].mean())

train_data['quarter_sum'] = train_data[['quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3',
       'quarter_Quarter4', 'quarter_Quarter5']].sum()
test_data['quarter_sum'] = test_data[['quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3',
       'quarter_Quarter4', 'quarter_Quarter5']].sum()

train_data['quarter_mean'] = train_data[['quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3',
       'quarter_Quarter4', 'quarter_Quarter5']].mean()
test_data['quarter_mean'] = test_data[['quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3',
       'quarter_Quarter4', 'quarter_Quarter5']].mean()

train_data['day_sum'] = train_data[['day_Monday', 'day_Saturday', 'day_Sunday',
       'day_Thursday', 'day_Tuesday', 'day_Wednesday']].sum()
test_data['day_sum'] = train_data[['day_Monday', 'day_Saturday', 'day_Sunday',
       'day_Thursday', 'day_Tuesday', 'day_Wednesday']].sum()

In [47]:
test_data['score'] = 0

In [49]:
model = lgb.LGBMRegressor(max_depth=10)
model.fit(train_data.drop(['id', 'score'], axis=1), train_data['score'])
test_data['score'] += model.predict(test_data.drop(['id', 'score'], axis=1))

model = lgb.LGBMRegressor(max_depth=7, min_child_samples=4)
model.fit(train_data.drop(['id', 'score'], axis=1), train_data['score'])
test_data['score'] += model.predict(test_data.drop(['id', 'score'], axis=1))

model = lgb.LGBMRegressor(max_depth=5, random_state=233)
model.fit(train_data.drop(['id', 'score'], axis=1), train_data['score'])
test_data['score'] += model.predict(test_data.drop(['id', 'score'], axis=1))

test_data['score'] /= 3

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005821 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1233
[LightGBM] [Info] Number of data points in the train set: 144000, number of used features: 26
[LightGBM] [Info] Start training from score 53.937500
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly 

In [50]:
test_data[['id', 'score']].to_csv('lgb.csv', index=None)